# Teken Pretrained Resnet as Feature Extractor

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import os
import numpy as np
from tqdm import tqdm

# Set device to GPU 1
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# Paths
DATASET_PATH = "dataset"
BATCH_SIZE = 32
NUM_WORKERS = 4

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
def get_dataloader(split):
    dataset_paths = []
    for dataset in ["Data Set 1", "Data Set 2", "Data Set 3", "Data Set 4"]:
        dataset_paths.append(os.path.join(DATASET_PATH, dataset, split))
    
    datasets_list = [datasets.ImageFolder(path, transform=transform) for path in dataset_paths]
    combined_dataset = torch.utils.data.ConcatDataset(datasets_list)
    
    return DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

train_loader = get_dataloader("train")
val_loader = get_dataloader("validation")
test_loader = get_dataloader("test")

# Fingerprint Extraction Model (ResNet-18)
class FingerprintExtractor(nn.Module):
    def __init__(self):
        super(FingerprintExtractor, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Identity()  # Remove final layer

    def forward(self, x):
        return self.model(x)

fingerprint_extractor = FingerprintExtractor().to(device)

# Extract Fingerprints
def extract_fingerprints(dataloader, model):
    model.eval()
    fingerprints = []
    labels = []
    with torch.no_grad():
        for images, targets in tqdm(dataloader, desc="Extracting Fingerprints"):
            images = images.to(device)
            features = model(images).cpu().numpy()
            fingerprints.append(features)
            labels.append(targets.numpy())
    return np.vstack(fingerprints), np.hstack(labels)

train_fingerprints, train_labels = extract_fingerprints(train_loader, fingerprint_extractor)
val_fingerprints, val_labels = extract_fingerprints(val_loader, fingerprint_extractor)
test_fingerprints, test_labels = extract_fingerprints(test_loader, fingerprint_extractor)

# Save fingerprints
np.save("train_fingerprints.npy", train_fingerprints)
np.save("train_labels.npy", train_labels)
np.save("val_fingerprints.npy", val_fingerprints)
np.save("val_labels.npy", val_labels)
np.save("test_fingerprints.npy", test_fingerprints)
np.save("test_labels.npy", test_labels)

# Classifier Model (MLP)
class FingerprintClassifier(nn.Module):
    def __init__(self, input_dim):
        super(FingerprintClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2)  # Real vs Fake
        )

    def forward(self, x):
        return self.fc(x)

classifier = FingerprintClassifier(input_dim=train_fingerprints.shape[1]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

# Convert fingerprints to PyTorch tensors
def get_tensor_data(fingerprints, labels):
    return torch.tensor(fingerprints, dtype=torch.float32).to(device), torch.tensor(labels, dtype=torch.long).to(device)

train_X, train_Y = get_tensor_data(train_fingerprints, train_labels)
val_X, val_Y = get_tensor_data(val_fingerprints, val_labels)
test_X, test_Y = get_tensor_data(test_fingerprints, test_labels)

# Train Classifier
def train_classifier(model, optimizer, criterion, epochs=20):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(train_X)
        loss = criterion(outputs, train_Y)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

train_classifier(classifier, optimizer, criterion)

# Evaluate Classifier
def evaluate_classifier(model, X, Y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).argmax(dim=1)
        accuracy = (predictions == Y).float().mean()
        print(f"Accuracy: {accuracy.item() * 100:.2f}%")

print("Validation Set:")
evaluate_classifier(classifier, val_X, val_Y)
print("Test Set:")
evaluate_classifier(classifier, test_X, test_Y)


/data/home/arunkumar12/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/home/arunkumar12/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting Fingerprints: 100%|██████████| 654/654 [00:21<00:00, 29.99it/s]


Epoch 1/20, Loss: 0.6972
Epoch 2/20, Loss: 0.6945
Epoch 3/20, Loss: 0.6919
Epoch 4/20, Loss: 0.6893
Epoch 5/20, Loss: 0.6869
Epoch 6/20, Loss: 0.6844
Epoch 7/20, Loss: 0.6820
Epoch 8/20, Loss: 0.6795
Epoch 9/20, Loss: 0.6771
Epoch 10/20, Loss: 0.6747
Epoch 11/20, Loss: 0.6722
Epoch 12/20, Loss: 0.6698
Epoch 13/20, Loss: 0.6674
Epoch 14/20, Loss: 0.6649
Epoch 15/20, Loss: 0.6625
Epoch 16/20, Loss: 0.6601
Epoch 17/20, Loss: 0.6577
Epoch 18/20, Loss: 0.6553
Epoch 19/20, Loss: 0.6529
Epoch 20/20, Loss: 0.6505
Validation Set:
Accuracy: 66.57%
Test Set:
Accuracy: 58.47%


# Finetuning on pretrained resnet as classifier

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import os

# Set device to GPU 1
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# Paths
DATASET_PATH = "dataset"
BATCH_SIZE = 32
NUM_WORKERS = 4

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
def get_dataloader(split):
    dataset_paths = []
    for dataset in ["Data Set 1", "Data Set 2", "Data Set 3", "Data Set 4"]:
        dataset_paths.append(os.path.join(DATASET_PATH, dataset, split))
    
    datasets_list = [datasets.ImageFolder(path, transform=transform) for path in dataset_paths]
    combined_dataset = torch.utils.data.ConcatDataset(datasets_list)
    
    return DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

train_loader = get_dataloader("train")
val_loader = get_dataloader("validation")
test_loader = get_dataloader("test")

# ResNet Model for Fake Image Detection
class FakeImageClassifier(nn.Module):
    def __init__(self):
        super(FakeImageClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, 2)  # Real vs Fake
    
    def forward(self, x):
        return self.model(x)

model = FakeImageClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Train Model
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

train_model(model, train_loader, criterion, optimizer)

# Evaluate Model
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    print(f"Accuracy: {100 * correct / total:.2f}%")

print("Validation Set:")
evaluate_model(model, val_loader)
print("Test Set:")
evaluate_model(model, test_loader)


Epoch 1/20, Loss: 0.0917
Epoch 2/20, Loss: 0.0432
Epoch 3/20, Loss: 0.0305
Epoch 4/20, Loss: 0.0232
Epoch 5/20, Loss: 0.0199
Epoch 6/20, Loss: 0.0166
Epoch 7/20, Loss: 0.0145
Epoch 8/20, Loss: 0.0134
Epoch 9/20, Loss: 0.0114
Epoch 10/20, Loss: 0.0110
Epoch 11/20, Loss: 0.0094
Epoch 12/20, Loss: 0.0093
Epoch 13/20, Loss: 0.0077
Epoch 14/20, Loss: 0.0084
Epoch 15/20, Loss: 0.0068
Epoch 16/20, Loss: 0.0071
Epoch 17/20, Loss: 0.0067
Epoch 18/20, Loss: 0.0060
Epoch 19/20, Loss: 0.0057
Epoch 20/20, Loss: 0.0054
Validation Set:
Accuracy: 98.18%
Test Set:
Accuracy: 93.59%
